In [136]:
using StatsBase
include("bitboard.jl")
rowLeftTable, rowRightTable, scoreTable, squareTable, mono1Table, mono2Table = initTables();

In [137]:
function mcts(board::UInt64, d, Δt)
    t0 = time()
    Q = Dict()
    N = Dict()
    T = Set()
    while time() - t0 < Δt
        simulate(board, d, Q, N,T)
    end
    astar, qstar = 0, 0
    for a in randperm(4) - 1
        if board != executeMove(a,board)
            q = Q[board, a]
            if q > qstar
                qstar = q
                astar = a
            end
        end
    end
    return astar
end

mcts (generic function with 1 method)

In [138]:
function simulate(board::UInt64,d,Q,N,T)
    if d == 0 || gameOver(board)
        return 0
    end
    if !(board in T)
        for a in randperm(4) - 1
            cboard = executeMove(a,board)
            if board != cboard
                Q[board,a] = scoreBoard(cboard)
                N[board,a] = 1
            end
        end
        push!(T,board)
        return rollout(board,d)
    end
    a = selectAction(board,Q,N)
    if a == -1
        return -1
    end
    cboard = executeMove(a,board)
    cboard = insertRandTile(cboard)
    r = scoreBoard(cboard)
    q = r + .95*simulate(cboard,d-1,Q,N,T)
    N[board,a] += 1
    Q[board,a] += (q - Q[board,a])/N[board,a]
    return q
end

simulate (generic function with 1 method)

In [139]:
function selectAction(board::UInt64,Q,N)
    l = numMoves(board)
    if l == 0
        return 0
    end
    args = zeros(l)
    vals = zeros(l)
    Ns = 0
    for a in 0:3
        if executeMove(a,board) != board
            Ns += N[board,a]
        end
    end
    i = 1
    for a in 0:3
        if executeMove(a,board) != board
            args[i] = a
            vals[i] = Q[board,a] + 100*sqrt(log(Ns)/N[board,a])
            i+=1
        end
    end
    val, id = findmax(vals)
    return args[id]
end

selectAction (generic function with 1 method)

In [140]:
function rollout(board::UInt64, d)
    l = numMoves(board)
    if d == 0 || l == 0 
        return scoreBoard(board)
    end
    args = zeros(l)
    vals = zeros(l)
    i = 1
    for a in 0:3
        if executeMove(a,board) != board
            args[i] = a
            vals[i] = scoreBoard(executeMove(a,board))
            i+=1
        end
    end
    
    a = sample(args, Weights(vals))
    
    cboard = executeMove(a,board)
    cboard = insertRandTile(cboard)
    
    return scoreBoard(cboard) + .75*rollout(cboard,d-1)
end

rollout (generic function with 1 method)

In [143]:
function mdp(board::UInt64)
    return mcts(board,4,.3)
end

mdp (generic function with 1 method)

In [144]:
tic()
n = 100
s = zeros(n,2)
for i in 1:n
    tic()
    val, score = runGame(mdp, false)
    println(i, " ", val, " ", score)
    s[i,1] = val
    s[i,2] = score
    toc()
end
toc()
println(mean(s[:,2]))
println(std(s[:,2]))

s = convert(DataFrame, s)
CSV.write("mcts4.csv", s)

3 4
4×4 Array{Int32,2}:
 0  0  0  4
 0  0  0  0
 0  0  0  2
 0  0  0  2
0 8
4×4 Array{Int32,2}:
 0  0  0  4
 0  0  0  4
 0  0  2  0
 0  0  0  0
1 16
4×4 Array{Int32,2}:
 0  2  0  0
 0  0  0  0
 0  0  0  0
 0  0  2  8
1 20
4×4 Array{Int32,2}:
 4  0  0  0
 0  0  0  0
 0  0  0  0
 0  2  2  8
2 24
4×4 Array{Int32,2}:
 4  0  0  0
 0  0  0  0
 0  0  0  0
 4  8  2  0
1 32
4×4 Array{Int32,2}:
 0  0  0  0
 0  0  0  0
 2  0  0  0
 8  8  2  0
2 48
4×4 Array{Int32,2}:
  0  0  0  0
  0  0  0  0
  2  2  0  0
 16  2  0  0
1 52
4×4 Array{Int32,2}:
  0  0  0  0
  0  0  0  0
  2  0  0  2
 16  4  0  0
3 56
4×4 Array{Int32,2}:
 0  0   0  0
 0  0   0  0
 0  0   2  4
 0  0  16  4
1 64
4×4 Array{Int32,2}:
 0  0   0  0
 0  0   0  0
 0  2   2  0
 0  0  16  8
2 68
4×4 Array{Int32,2}:
  0  0  0  0
  0  0  0  0
  4  0  0  0
 16  8  2  0
0 68
4×4 Array{Int32,2}:
  4  8  2  0
 16  0  0  0
  0  0  0  0
  0  0  0  2
1 68
4×4 Array{Int32,2}:
  0  0  0  0
  0  0  0  0
  4  2  0  0
 16  8  2  2
2 72
4×4 Array{Int32,2}:


1 1188
4×4 Array{Int32,2}:
 0  0   0    0
 0  0   0   64
 4  0   2  128
 2  2  16   16
3 1224
4×4 Array{Int32,2}:
 0  0  0    0
 0  0  0   64
 2  4  2  128
 0  0  4   32
1 1224
4×4 Array{Int32,2}:
 2  0  0    0
 0  0  0   64
 0  0  2  128
 2  4  4   32
1 1228
4×4 Array{Int32,2}:
 0  2  0    0
 0  0  0   64
 0  0  2  128
 4  4  4   32
0 1228
4×4 Array{Int32,2}:
 4  2  2   64
 0  4  4  128
 0  0  0   32
 0  2  0    0
3 1240
4×4 Array{Int32,2}:
 2  4  4   64
 0  0  8  128
 0  0  0   32
 0  0  0    2
3 1248
4×4 Array{Int32,2}:
 0  2  8   64
 0  0  8  128
 2  0  0   32
 0  0  0    2
0 1264
4×4 Array{Int32,2}:
 2  2  16   64
 0  0   0  128
 0  0   0   32
 0  2   0    2
3 1272
4×4 Array{Int32,2}:
 0  4  16   64
 0  0   0  128
 0  0   2   32
 0  0   0    4
0 1272
4×4 Array{Int32,2}:
 2  4  16   64
 0  0   2  128
 0  0   0   32
 0  0   0    4
1 1272
4×4 Array{Int32,2}:
 0  0   0   64
 0  0   2  128
 0  0  16   32
 2  4   2    4
0 1272
4×4 Array{Int32,2}:
 2  4   2   64
 0  0  16  128
 0  0   2 

3 2720
4×4 Array{Int32,2}:
 0  0   8    8
 0  0   8   16
 2  2  16  128
 0  0   8  256
3 2740
4×4 Array{Int32,2}:
 0  0   0   16
 0  0   8   16
 2  4  16  128
 0  0   8  256
1 2772
4×4 Array{Int32,2}:
 0  0   0    0
 0  2   8   32
 0  0  16  128
 2  4   8  256
1 2772
4×4 Array{Int32,2}:
 0  0   2    0
 0  0   8   32
 0  2  16  128
 2  4   8  256
3 2772
4×4 Array{Int32,2}:
 0  0   0    2
 0  2   8   32
 0  2  16  128
 2  4   8  256
1 2776
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   8   32
 2  4  16  128
 2  4   8  256
1 2788
4×4 Array{Int32,2}:
 0  2   0    2
 0  0   8   32
 0  0  16  128
 4  8   8  256
3 2808
4×4 Array{Int32,2}:
 0  0   0    4
 0  0   8   32
 0  2  16  128
 0  4  16  256
1 2840
4×4 Array{Int32,2}:
 0  0   0    4
 0  2   0   32
 0  2   8  128
 0  4  32  256
1 2844
4×4 Array{Int32,2}:
 0  0   0    4
 0  2   0   32
 0  4   8  128
 0  4  32  256
1 2852
4×4 Array{Int32,2}:
 2  0   0    4
 0  0   0   32
 0  2   8  128
 0  8  32  256
3 2852
4×4 Array{Int32,2}:
 0  0   2    4
 

3 4704
4×4 Array{Int32,2}:
 0  0  8   32
 2  2  8   32
 0  0  2   64
 0  0  0  512
1 4784
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   0   64
 0  0  16   64
 2  2   2  512
3 4792
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   4   64
 0  0  16   64
 0  2   4  512
1 4920
4×4 Array{Int32,2}:
 0  0   2    0
 0  0   4    2
 0  0  16  128
 0  2   4  512
3 4920
4×4 Array{Int32,2}:
 2  0   0    2
 0  0   4    2
 0  0  16  128
 0  2   4  512
1 4924
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   4    4
 0  0  16  128
 2  2   4  512
3 4936
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   0    8
 0  2  16  128
 0  4   4  512
3 4944
4×4 Array{Int32,2}:
 0  0   0    2
 0  0   0    8
 2  2  16  128
 0  0   8  512
3 4948
4×4 Array{Int32,2}:
 0  2   0    2
 0  0   0    8
 0  4  16  128
 0  0   8  512
3 4952
4×4 Array{Int32,2}:
 0  0   0    4
 0  0   0    8
 2  4  16  128
 0  0   8  512
1 4952
4×4 Array{Int32,2}:
 0  0   0    4
 0  0   0    8
 0  2  16  128
 2  4   8  512
0 4952
4×4 Array{Int32,2}:
 2  2  16    4
 0  4

3 6320
4×4 Array{Int32,2}:
 0  0   2   16
 0  0   2   64
 0  2   8  256
 0  2  16  512
0 6328
4×4 Array{Int32,2}:
 0  4   4   16
 0  0   8   64
 0  0  16  256
 0  2   0  512
3 6336
4×4 Array{Int32,2}:
 0  0   8   16
 0  0   8   64
 0  0  16  256
 0  2   2  512
3 6340
4×4 Array{Int32,2}:
 0  0   8   16
 0  0   8   64
 0  0  16  256
 0  2   4  512
1 6356
4×4 Array{Int32,2}:
 0  0   0   16
 0  0  16   64
 0  0  16  256
 2  2   4  512
3 6364
4×4 Array{Int32,2}:
 0  0   0   16
 0  4  16   64
 0  0  16  256
 0  4   4  512
3 6372
4×4 Array{Int32,2}:
 2  0   0   16
 0  4  16   64
 0  0  16  256
 0  0   8  512
1 6404
4×4 Array{Int32,2}:
 0  0   0   16
 0  0   0   64
 0  2  32  256
 2  4   8  512
0 6404
4×4 Array{Int32,2}:
 2  2  32   16
 2  4   8   64
 0  0   0  256
 0  0   0  512
1 6408
4×4 Array{Int32,2}:
 0  0   0   16
 0  2   0   64
 0  2  32  256
 4  4   8  512
3 6416
4×4 Array{Int32,2}:
 0  2   0   16
 0  0   2   64
 0  2  32  256
 0  8   8  512
3 6432
4×4 Array{Int32,2}:
 0  0   2   16
 

...

3 29876
4×4 Array{Int32,2}:
  0   0    0     2
  2   4    8   128
  2   0   16   512
 16  64  512  2048
1 29884
4×4 Array{Int32,2}:
  0   4    0     2
  0   0    8   128
  4   4   16   512
 16  64  512  2048
1 29892
4×4 Array{Int32,2}:
  0   0    0     2
  0   2    8   128
  4   8   16   512
 16  64  512  2048
2 29892
4×4 Array{Int32,2}:
  2   0    2     0
  2   8  128     0
  4   8   16   512
 16  64  512  2048
1 29912
4×4 Array{Int32,2}:
  0   0    2     2
  4   0  128     0
  4  16   16   512
 16  64  512  2048
3 29952
4×4 Array{Int32,2}:
  0   0    0     4
  0   4    4   128
  0   4   32   512
 16  64  512  2048
1 29960
4×4 Array{Int32,2}:
  0   0    2     4
  0   0    4   128
  0   8   32   512
 16  64  512  2048
2 29960
4×4 Array{Int32,2}:
  2    4    0     0
  4  128    0     2
  8   32  512     0
 16   64  512  2048
3 29960
4×4 Array{Int32,2}:
  0   0    2     4
  0   4  128     2
  2   8   32   512
 16  64  512  2048
2 29960
4×4 Array{Int32,2}:
  2    4    2     0
  4  128    

1 32324
4×4 Array{Int32,2}:
  0    0     0     0
  0    0     0     8
  2    4     2   128
 16  256  1024  2048
2 32324
4×4 Array{Int32,2}:
  0    0     0     0
  8    0     0     2
  2    4     2   128
 16  256  1024  2048
3 32324
4×4 Array{Int32,2}:
  0    0     2     0
  0    0     8     2
  2    4     2   128
 16  256  1024  2048
3 32328
4×4 Array{Int32,2}:
  0    0     0     2
  4    0     8     2
  2    4     2   128
 16  256  1024  2048
3 32332
4×4 Array{Int32,2}:
  4    0     0     2
  0    4     8     2
  2    4     2   128
 16  256  1024  2048
2 32332
4×4 Array{Int32,2}:
  4    2     0     0
  4    8     2     2
  2    4     2   128
 16  256  1024  2048
1 32344
4×4 Array{Int32,2}:
  0    2     0     0
  8    8     2     2
  2    4     4   128
 16  256  1024  2048
3 32372
4×4 Array{Int32,2}:
  2    0     0     2
  0    0    16     4
  0    2     8   128
 16  256  1024  2048
3 32376
4×4 Array{Int32,2}:
  0    0     2     4
  0    0    16     4
  0    2     8   128
 16  256  102

1 33512
4×4 Array{Int32,2}:
  0    0     2     0
  4    0     0     2
  4   16    16   256
 32  256  1024  2048
1 33520
4×4 Array{Int32,2}:
  0    0     0     0
  2    0     2     2
  8   16    16   256
 32  256  1024  2048
3 33556
4×4 Array{Int32,2}:
  2    0     0     0
  0    0     2     4
  0    8    32   256
 32  256  1024  2048
3 33556
4×4 Array{Int32,2}:
  0    2     0     2
  0    0     2     4
  0    8    32   256
 32  256  1024  2048
3 33560
4×4 Array{Int32,2}:
  0    0     2     4
  0    0     2     4
  0    8    32   256
 32  256  1024  2048
1 33572
4×4 Array{Int32,2}:
  0    0     0     0
  2    0     4     8
  0    8    32   256
 32  256  1024  2048
1 33572
4×4 Array{Int32,2}:
  0    0     0     2
  0    0     4     8
  2    8    32   256
 32  256  1024  2048
2 33572
4×4 Array{Int32,2}:
  2    0     2     0
  4    8     0     0
  2    8    32   256
 32  256  1024  2048
1 33588
4×4 Array{Int32,2}:
  2    0     2     0
  4    0     2     0
  2   16    32   256
 32  256  102

3 34292
4×4 Array{Int32,2}:
  4    2     8     2
  2    2    16    64
 16   32    64   256
 32  256  1024  2048
2 34296
4×4 Array{Int32,2}:
  4    2     8     2
  4   16    64     2
 16   32    64   256
 32  256  1024  2048
1 34436
4×4 Array{Int32,2}:
  0    2     2     0
  8   16     8     4
 16   32   128   256
 32  256  1024  2048
3 34440
4×4 Array{Int32,2}:
  0    0     2     4
  8   16     8     4
 16   32   128   256
 32  256  1024  2048
1 34448
4×4 Array{Int32,2}:
  0    0     2     2
  8   16     8     8
 16   32   128   256
 32  256  1024  2048
3 34468
4×4 Array{Int32,2}:
  0    2     0     4
  0    8    16    16
 16   32   128   256
 32  256  1024  2048
3 34500
4×4 Array{Int32,2}:
  0    0     2     4
  0    2     8    32
 16   32   128   256
 32  256  1024  2048
2 34504
4×4 Array{Int32,2}:
  2    4     0     0
  2    8    32     4
 16   32   128   256
 32  256  1024  2048
1 34508
4×4 Array{Int32,2}:
  0    4     2     0
  4    8    32     4
 16   32   128   256
 32  256  102

3 35304
4×4 Array{Int32,2}:
  2    4     4     8
  8   32    64     2
 32   64   128   256
 64  256  1024  2048
2 35312
4×4 Array{Int32,2}:
  2    8     8     2
  8   32    64     2
 32   64   128   256
 64  256  1024  2048
1 35316
4×4 Array{Int32,2}:
  2    8     8     2
  8   32    64     4
 32   64   128   256
 64  256  1024  2048
2 35332
4×4 Array{Int32,2}:
  2   16     2     2
  8   32    64     4
 32   64   128   256
 64  256  1024  2048
3 35336
4×4 Array{Int32,2}:
  2    2    16     4
  8   32    64     4
 32   64   128   256
 64  256  1024  2048
1 35344
4×4 Array{Int32,2}:
  2    2    16     2
  8   32    64     8
 32   64   128   256
 64  256  1024  2048
2 35348
4×4 Array{Int32,2}:
  4   16     2     2
  8   32    64     8
 32   64   128   256
 64  256  1024  2048
3 35352
4×4 Array{Int32,2}:
  2    4    16     4
  8   32    64     8
 32   64   128   256
 64  256  1024  2048
elapsed time: 368.894210107 seconds
2048.0
NaN


In [110]:
function mdp(board::UInt64)
    return mcts(board,5,.3)
end

1-element Array{Float64,1}:
 0.0

In [86]:
tic()
n = 100
s = zeros(n,2)
for i in 1:n
    tic()
    val, score = runGame(mdp, false)
    println(i, " ", val, " ", score)
    s[i,1] = val
    s[i,2] = score
    toc()
end
toc()
println(mean(s[:,2]))
println(std(s[:,2]))

s = convert(DataFrame, s)
CSV.write("mcts5.csv", s)

elapsed time: 0.100314816 seconds


0.100314816

In [145]:
using StatsBase
include("bitboard.jl")
rowLeftTable, rowRightTable, scoreTable, squareTable, mono1Table, mono2Table = initTables();

In [146]:
function mcts(board::UInt64, d, Δt)
    t0 = time()
    Q = Dict()
    N = Dict()
    T = Set()
    while time() - t0 < Δt
        simulate(board, d, Q, N,T)
    end
    astar, qstar = 0, 0
    for a in randperm(4) - 1
        if board != executeMove(a,board)
            q = Q[board, a]
            if q > qstar
                qstar = q
                astar = a
            end
        end
    end
    return astar
end

mcts (generic function with 1 method)

In [147]:
function simulate(board::UInt64,d,Q,N,T)
    if d == 0 || gameOver(board)
        return scoreBoard(board) 
    end
    if !(board in T)
        for a in randperm(4) - 1
            cboard = executeMove(a,board)
            if board != cboard
                Q[board,a] = 0
                N[board,a] = 1
            end
        end
        push!(T,board)
        return rollout(board,d)
    end
    a = selectAction(board,Q,N)
    if a == -1
        return -1
    end
    cboard = executeMove(a,board)
    cboard = insertRandTile(cboard)
    #r = scoreBoard(cboard)
    r = 0
    q = r + 1*simulate(cboard,d-1,Q,N,T)
    N[board,a] += 1
    Q[board,a] += (q - Q[board,a])/N[board,a]
    return q
end

simulate (generic function with 1 method)

In [148]:
function selectAction(board::UInt64,Q,N)
    l = numMoves(board)
    if l == 0
        return 0
    end
    args = zeros(l)
    vals = zeros(l)
    Ns = 0
    for a in 0:3
        if executeMove(a,board) != board
            Ns += N[board,a]
        end
    end
    i = 1
    for a in 0:3
        if executeMove(a,board) != board
            args[i] = a
            vals[i] = Q[board,a] + 10*sqrt(log(Ns)/N[board,a])
            i+=1
        end
    end
    val, id = findmax(vals)
    return args[id]
end

selectAction (generic function with 1 method)

In [149]:
function rollout(board::UInt64, d)
    l = numMoves(board)
    if d == 0 || l == 0 
        return scoreBoard(board)
    end
    args = zeros(l)
    vals = zeros(l)
    i = 1
    for a in 0:3
        if executeMove(a,board) != board
            args[i] = a
            vals[i] = scoreBoard(executeMove(a,board))
            i+=1
        end
    end
    
    a = sample(args, Weights(vals))
    
    cboard = executeMove(a,board)
    cboard = insertRandTile(cboard)
    
    #return scoreBoard(cboard) + .75*rollout(cboard,d-1)
    return rollout(cboard,d-1)
end

rollout (generic function with 1 method)

In [150]:
tic()
n = 1
s = zeros(n)
for i in 1:n
    s[i] = runGame(mdp, true)
end
toc()
println(mean(s))
println(std(s))

0 4
4×4 Array{Int32,2}:
 0  0  4  2
 0  0  2  0
 0  0  0  0
 0  0  0  0
1 4
4×4 Array{Int32,2}:
 0  0  0  2
 0  0  0  0
 0  0  4  0
 0  0  2  2
0 8
4×4 Array{Int32,2}:
 0  0  4  4
 0  0  2  2
 0  0  0  0
 0  0  0  0
1 8
4×4 Array{Int32,2}:
 0  0  0  0
 0  0  0  0
 0  0  4  4
 0  2  2  2
0 8
4×4 Array{Int32,2}:
 2  2  4  4
 0  0  2  2
 0  0  0  0
 0  0  0  0
1 8
4×4 Array{Int32,2}:
 0  0  0  2
 0  0  0  0
 0  0  4  4
 2  2  2  2
0 8
4×4 Array{Int32,2}:
 2  2  4  2
 2  0  2  4
 0  0  0  2
 0  0  0  0
0 12
4×4 Array{Int32,2}:
 4  2  4  2
 0  0  2  4
 0  0  0  2
 2  0  0  0
0 12
4×4 Array{Int32,2}:
 4  2  4  2
 2  0  2  4
 0  0  2  2
 0  0  0  0
0 16
4×4 Array{Int32,2}:
 4  2  4  2
 2  0  4  4
 0  0  0  2
 2  0  0  0
0 28
4×4 Array{Int32,2}:
 4  2  8  2
 4  0  0  4
 0  2  0  2
 0  0  0  0
0 40
4×4 Array{Int32,2}:
 8  4  8  2
 0  0  0  4
 0  0  0  2
 0  2  0  0
0 40
4×4 Array{Int32,2}:
 8  4  8  2
 2  2  0  4
 0  0  0  2
 0  0  0  0
1 40
4×4 Array{Int32,2}:
 2  0  0  0
 0  0  0  2
 8  4  0 

2 1088
4×4 Array{Int32,2}:
 128   4  2  0
  32  16  4  0
  32   4  0  0
   4   2  0  0
0 1152
4×4 Array{Int32,2}:
 128   4  2  0
  64  16  4  2
   4   4  0  0
   0   2  0  0
0 1152
4×4 Array{Int32,2}:
 128   4  2  2
  64  16  4  0
   4   4  0  0
   0   2  0  2
0 1160
4×4 Array{Int32,2}:
 128   4  2  4
  64  16  4  4
   4   4  0  0
   0   2  0  0
0 1168
4×4 Array{Int32,2}:
 128   4  2  8
  64  16  4  0
   4   4  0  0
   0   2  0  2
0 1168
4×4 Array{Int32,2}:
 128   4  2  8
  64  16  4  2
   4   4  0  0
   0   2  2  0
0 1168
4×4 Array{Int32,2}:
 128   4  2  8
  64  16  4  2
   4   4  2  0
   0   2  0  2
0 1172
4×4 Array{Int32,2}:
 128   4  2  8
  64  16  4  4
   4   4  2  2
   0   2  0  0
1 1172
4×4 Array{Int32,2}:
   2   4  0  0
 128  16  2  8
  64   4  4  4
   4   2  2  2
3 1184
4×4 Array{Int32,2}:
   0   2  2  4
 128  16  2  8
   0  64  4  8
   0   4  2  4
0 1204
4×4 Array{Int32,2}:
 128   2  4   4
   2  16  4  16
   0  64  2   4
   0   4  0   0
2 1212
4×4 Array{Int32,2}:
 128   2  8 

2 2536
4×4 Array{Int32,2}:
  8   8  4    0
  4  64  4  256
  4  16  4    0
 64   0  0    4
0 2552
4×4 Array{Int32,2}:
  8   8  8  256
  8  64  4    4
 64  16  0    0
  2   0  0    0
2 2580
4×4 Array{Int32,2}:
 16   8  256  0
  8  64    8  4
 64  16    0  0
  2   0    0  0
3 2580
4×4 Array{Int32,2}:
 0  16   8  256
 8  64   8    4
 0   0  64   16
 0   0   2    2
0 2596
4×4 Array{Int32,2}:
 8  16  16  256
 0  64  64    4
 0   0   2   16
 0   0   2    2
0 2600
4×4 Array{Int32,2}:
 8  16  16  256
 0  64  64    4
 0   2   4   16
 0   0   0    2
2 2760
4×4 Array{Int32,2}:
   8  32  256  0
 128   4    0  0
   2   4   16  0
   2   2    0  0
0 2772
4×4 Array{Int32,2}:
   8  32  256  0
 128   8   16  0
   4   2    0  0
   2   0    0  0
1 2772
4×4 Array{Int32,2}:
   8   0    2  0
 128  32    0  0
   4   8  256  0
   2   2   16  0
2 2776
4×4 Array{Int32,2}:
   8   2    2  0
 128  32    0  0
   4   8  256  0
   4  16    0  0
0 2784
4×4 Array{Int32,2}:
   8   2    2  0
 128  32  256  0
   8   8    0

2 3960
4×4 Array{Int32,2}:
  16  32   8  256
 256  32  16    2
   4   2   0    0
   4   0   0    0
0 4032
4×4 Array{Int32,2}:
  16  64   8  256
 256   2  16    2
   8   2   0    0
   0   0   0    0
3 4032
4×4 Array{Int32,2}:
  16  64   8  256
 256   2  16    2
   0   2   8    2
   0   0   0    0
0 4040
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   0   0   8    0
   0   2   0    0
2 4040
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   8   0   0    0
   2   2   0    0
2 4044
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   8   0   0    0
   4   0   2    0
2 4044
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   8   0   2    0
   4   2   0    0
2 4044
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   8   2   2    0
   4   2   0    0
2 4048
4×4 Array{Int32,2}:
  16  64   8  256
 256   4  16    4
   8   4   0    0
   4   2   0    2
0 4056
4×4 Array{Int32,2}:
  16  64   8  256
 256   8  16    4
   8   2   0    2
   4   0   2    0
0 4056
4×4

0 5524
4×4 Array{Int32,2}:
  16   4  128  2
 512  32   64  8
   8  32    2  2
   2   2    0  0
3 5532
4×4 Array{Int32,2}:
  16   4  128  2
 512  32   64  8
   0   8   32  4
   0   2    0  4
3 5532
4×4 Array{Int32,2}:
  16   4  128  2
 512  32   64  8
   0   8   32  4
   2   0    2  4
2 5536
4×4 Array{Int32,2}:
  16   4  128  2
 512  32   64  8
   8  32    4  0
   4   4    0  2
2 5544
4×4 Array{Int32,2}:
  16   4  128  2
 512  32   64  8
   8  32    4  2
   8   2    0  0
0 5624
4×4 Array{Int32,2}:
  16   4  128  2
 512  64   64  8
  16   2    4  2
   0   2    0  0
2 5752
4×4 Array{Int32,2}:
  16    4  128  2
 512  128    8  0
  16    2    4  2
   2    0    0  2
2 5756
4×4 Array{Int32,2}:
  16    4  128  2
 512  128    8  0
  16    2    4  2
   4    2    0  0
3 5756
4×4 Array{Int32,2}:
 16    4  128  2
  0  512  128  8
 16    2    4  2
  0    2    4  2
1 6060
4×4 Array{Int32,2}:
  2    0    0  0
  0    4    0  2
  0  512  256  8
 32    4    8  4
2 6064
4×4 Array{Int32,2}:
   2    4  0  0

1 6940
4×4 Array{Int32,2}:
   4   32   2  0
  16  256   0  8
   8  128  16  4
 512    4   8  2
2 6940
4×4 Array{Int32,2}:
   4   32   2  2
  16  256   8  0
   8  128  16  4
 512    4   8  2
2 6944
4×4 Array{Int32,2}:
   4   32   4  0
  16  256   8  2
   8  128  16  4
 512    4   8  2
0 6944
4×4 Array{Int32,2}:
   4   32   4  2
  16  256   8  4
   8  128  16  2
 512    4   8  2
0 6948
4×4 Array{Int32,2}:
   4   32   4  2
  16  256   8  4
   8  128  16  4
 512    4   8  2
1 6956
4×4 Array{Int32,2}:
   4   32   4  2
  16  256   8  2
   8  128  16  8
 512    4   8  2
0 6960
4×4 Array{Int32,2}:
   4   32   4  4
  16  256   8  8
   8  128  16  2
 512    4   8  2
2 6984
4×4 Array{Int32,2}:
   4   32   8  2
  16  256  16  0
   8  128  16  2
 512    4   8  2
1 7020
4×4 Array{Int32,2}:
   4   32   2  0
  16  256   8  0
   8  128  32  2
 512    4   8  4
0 7024
4×4 Array{Int32,2}:
   4   32   2  2
  16  256   8  4
   8  128  32  4
 512    4   8  0
0 7032
4×4 Array{Int32,2}:
   4   32   2  2
  16  

0 9880
4×4 Array{Int32,2}:
  8     4  32  16
 64  1024  32   4
  2     2   8   2
  0     0   0   0
0 9944
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  2     2   0   2
  0     0   2   0
0 9944
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  2     2   2   2
  2     0   0   0
3 9952
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  0     0   4   4
  0     0   2   2
3 9964
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  0     0   2   8
  0     0   0   4
2 9964
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  2     8   2   0
  4     0   0   0
3 9964
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  2     2   8   2
  0     0   0   4
2 9968
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  4     8   2   0
  4     0   0   2
3 9972
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  0     4   8   2
  0     4   4   2
3 9980
4×4 Array{Int32,2}:
  8     4  64  16
 64  1024   8   4
  2     4   8   2
  0     0   8   2
0 10000
4×